In [1]:
from sklearn.datasets import load_wine
import pandas as pd

data = load_wine()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

print(df.head())

   alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0    14.23        1.71  2.43               15.6      127.0           2.80   
1    13.20        1.78  2.14               11.2      100.0           2.65   
2    13.16        2.36  2.67               18.6      101.0           2.80   
3    14.37        1.95  2.50               16.8      113.0           3.85   
4    13.24        2.59  2.87               21.0      118.0           2.80   

   flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
0        3.06                  0.28             2.29             5.64  1.04   
1        2.76                  0.26             1.28             4.38  1.05   
2        3.24                  0.30             2.81             5.68  1.03   
3        3.49                  0.24             2.18             7.80  0.86   
4        2.69                  0.39             1.82             4.32  1.04   

   od280/od315_of_diluted_wines  proline  target  
0          

In [2]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

scalers = {
    'min_max_scaler': MinMaxScaler(),
    'standard_scaler': StandardScaler(),
    'robust_scaler': RobustScaler()
}

scaled_data = {}
for scaler_name, scaler in scalers.items():
    scaled_data[scaler_name] = pd.DataFrame(scaler.fit_transform(df.drop('target', axis=1)), columns=df.columns[:-1])
# Выводим результаты масштабирования для всех масштабировщиков
for scaler_name, scaler_data in scaled_data.items():
    print(f"Результат {scaler_name}:")
    print(scaler_data.head())
    print()  # Добавляет пустую строку между результатами для улучшения читаемости

Результат min_max_scaler:
    alcohol  malic_acid       ash  alcalinity_of_ash  magnesium  \
0  0.842105    0.191700  0.572193           0.257732   0.619565   
1  0.571053    0.205534  0.417112           0.030928   0.326087   
2  0.560526    0.320158  0.700535           0.412371   0.336957   
3  0.878947    0.239130  0.609626           0.319588   0.467391   
4  0.581579    0.365613  0.807487           0.536082   0.521739   

   total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
0       0.627586    0.573840              0.283019         0.593060   
1       0.575862    0.510549              0.245283         0.274448   
2       0.627586    0.611814              0.320755         0.757098   
3       0.989655    0.664557              0.207547         0.558360   
4       0.627586    0.495781              0.490566         0.444795   

   color_intensity       hue  od280/od315_of_diluted_wines   proline  
0         0.372014  0.455285                      0.970696  0.561341  
1 

In [3]:
#3.1
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

df_filtered = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
print("Данные после удаления выбросов:")
print(df_filtered.shape)

Данные после удаления выбросов:
(161, 14)


In [4]:
#3.2
for col in df.columns:
    col_Q1 = df[col].quantile(0.25)
    col_Q3 = df[col].quantile(0.75)
    col_IQR = col_Q3 - col_Q1
    lower_bound = col_Q1 - 1.5 * col_IQR
    upper_bound = col_Q3 + 1.5 * col_IQR
    df.loc[df[col] < lower_bound, col] = df[col].mean()
    df.loc[df[col] > upper_bound, col] = df[col].mean()

print("Данные после замены выбросов на среднее значение:")
print(df.head())

Данные после замены выбросов на среднее значение:
   alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0    14.23        1.71  2.43               15.6      127.0           2.80   
1    13.20        1.78  2.14               11.2      100.0           2.65   
2    13.16        2.36  2.67               18.6      101.0           2.80   
3    14.37        1.95  2.50               16.8      113.0           3.85   
4    13.24        2.59  2.87               21.0      118.0           2.80   

   flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
0        3.06                  0.28             2.29             5.64  1.04   
1        2.76                  0.26             1.28             4.38  1.05   
2        3.24                  0.30             2.81             5.68  1.03   
3        3.49                  0.24             2.18             7.80  0.86   
4        2.69                  0.39             1.82             4.32  1.04   

   od280/od3

In [5]:
df['alcohol_category'] = pd.cut(df['alcohol'], bins=3, labels=["low", "medium", "high"])
print("Преобразованный признак 'alcohol' в категории:")
print(df['alcohol_category'].head())

Преобразованный признак 'alcohol' в категории:
0      high
1    medium
2    medium
3      high
4    medium
Name: alcohol_category, dtype: category
Categories (3, object): ['low' < 'medium' < 'high']


In [6]:
#5.1
from sklearn.feature_selection import SelectKBest, f_classif

# Отбираем три лучших признака
selector = SelectKBest(f_classif, k=3)
selector.fit(df.drop(['target', 'alcohol_category'], axis=1), df['target'])
df_filtered_features = df.drop(['target', 'alcohol_category'], axis=1).columns[selector.get_support()]

print("Отобранные признаки методом фильтрации:")
print(df_filtered_features)

Отобранные признаки методом фильтрации:
Index(['flavanoids', 'od280/od315_of_diluted_wines', 'proline'], dtype='object')


In [7]:
#5.2
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

# Рекурсивное устранение признаков
rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select=3)
rfe.fit(df.drop(['target', 'alcohol_category'], axis=1), df['target'])

print("Признаки, отобранные с помощью RFE:")
print(df.drop(['target', 'alcohol_category'], axis=1).columns[rfe.support_])

Признаки, отобранные с помощью RFE:
Index(['flavanoids', 'color_intensity', 'proline'], dtype='object')


In [8]:
#5.3
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

# Отбор признаков с помощью регуляризации в модели логистической регрессии
selector = SelectFromModel(estimator=LogisticRegression(penalty="l1", solver='liblinear'), max_features=3)
selector.fit(df.drop(['target', 'alcohol_category'], axis=1), df['target'])

print("Признаки, отобранные методом вложений:")
print(df.drop(['target', 'alcohol_category'], axis=1).columns[selector.get_support()])

Признаки, отобранные методом вложений:
Index(['flavanoids', 'color_intensity', 'hue'], dtype='object')
